# Training and hosting SageMaker Models using the Apache MXNet Module API

The **SageMaker Python SDK** makes it easy to train and deploy MXNet models. In this example, we train a simple neural network using the Apache MXNet [Module API](https://mxnet.apache.org/api/python/module/module.html) and the driving score dataset.

### Setup

First we need to define a few variables that will be needed later in the example.

In [ ]:
from sagemaker import get_execution_role
from sagemaker.session import Session

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = Session().default_bucket()

# Location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://{}/customcode/mxnet'.format(bucket)

# Location where results of model training are saved.
model_artifacts_location = 's3://{}/artifacts'.format(bucket)

# IAM execution role that gives SageMaker access to resources in your AWS account.
# We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

### The training script

The ``maintenance.py`` script provides all the code we need for training and hosting a SageMaker model. The script also checkpoints the model at the end of every epoch and saves the model graph, params and optimizer state in the folder `/opt/ml/checkpoints`. If the folder path does not exist then it will skip checkpointing.

In [ ]:
print(bucket)

In [ ]:
!cat train.py

### SageMaker's MXNet estimator class

The SageMaker ```MXNet``` estimator allows us to run single machine or distributed training in SageMaker, using CPU or GPU-based instances.

When we create the estimator, we pass in the filename of our training script, the name of our IAM execution role, and the S3 locations we defined in the setup section. We also provide a few other parameters. ``train_instance_count`` and ``train_instance_type`` determine the number and type of SageMaker instances that will be used for the training job. The ``hyperparameters`` parameter is a ``dict`` of values that will be passed to your training script -- you can see how to access these values in the ``train.py`` script above.

For this example, we will choose one ``ml.m4.xlarge`` instance.

In [ ]:
from sagemaker.mxnet import MXNet

drvscore_estimator = MXNet(entry_point='train.py',
                           role=role,
                           output_path=model_artifacts_location,
                           code_location=custom_code_upload_location,
                           train_instance_count=1,
                           train_instance_type='ml.m4.xlarge',
                           framework_version='1.2.1',
                           py_version='py2')

### Running the Training Job

After we've constructed our MXNet object, we can fit it using data stored in S3. Below we run SageMaker training on two input channels: **train** and **test**.

During training, SageMaker makes this data stored in S3 available in the local filesystem where the script is running. The ```train.py``` script simply loads the train and test data from disk.

In [ ]:
%%time
import boto3

region = boto3.Session().region_name
data_location = 's3://sagemaker-connectedcar/driver-scores-dataset/data.pickle'
label_location = 's3://sagemaker-connectedcar/driver-scores-dataset/label.pickle'

In [ ]:
drvscore_estimator.fit({'data': data_location, 'label': label_location},job_name='connectedcar-drvscore')

# (Optional) Creating an inference Endpoint

After training, we use the ``MXNet estimator`` object to build and deploy an ``MXNetPredictor``. This creates a Sagemaker **Endpoint** -- a hosted prediction service that we can use to perform inference. 

The arguments to the ``deploy`` function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances. Here we will deploy the model to a single ``ml.m4.xlarge`` instance.

In [ ]:
#predictor = drvscore_estimator.deploy(initial_instance_count=1,
#                                   instance_type='ml.m4.xlarge')

# (Optional) Making an inference request

Now that our Endpoint is deployed and we have a ``predictor`` object, we can use it to run inference.

In [ ]:
# Load data you want to run inference
#data = ...
#response = predictor.predict(data)

# (Optional) Delete the Endpoint

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
#print("Endpoint name: " + predictor.endpoint)
#predictor.delete_endpoint()